In [2]:
import os 
from config import  BRAIN_REGION_PAIRS, SUBMITIT_PARAMS, extraction_parameters
import pandas as pd
import numpy as np
import mne
import submitit
import gc
import config
import time
from functools import partial

from data_extraction import data_extraction


data_extraction_with_params = partial(data_extraction, **extraction_parameters)
##########################
# Submit jobs
##########################

if len(BRAIN_REGION_PAIRS) >1 :
    raise ValueError("BRAIN_REGION_PAIRS should have only one pair of brain regions for extraction")

region1 = BRAIN_REGION_PAIRS[0][0]
region2 = BRAIN_REGION_PAIRS[0][1]

table_path = os.path.join(os.getcwd(), f'data/eid_probe_info_{region1}_{region2}.csv')
if not os.path.isfile(table_path):
    print(f"eid_probe_info_{region1}_{region2}.csv does not exist at {os.getcwd()}")
    print("use eid_probe_info_table.py to generate the table")
    raise ValueError(f"table does not exist at {os.getcwd()}")

# Load the table
df = pd.read_csv(table_path)
job_args = []

for i, row in df.iterrows():
    # if i>0:
    #     break
    eid = row['eid']
    probe_label_1 = row[f'label_{region1}'][2:-2]
    probe_label_2 = row[f'label_{region2}'][2:-2]
    probe_labels = [probe_label_1, probe_label_2]
    probe_labels = list(set(probe_labels))
    

    for probe_label in probe_labels:
        job_args.append((eid, probe_label))

In [22]:
eids = os.listdir('/mnt/data/AdaptiveControl/IBLrawdata/eid_data/') 

In [25]:
dsets = one.list_datasets()

In [27]:
trial = one.load_dataset(eid, 'alf/_ibl_trials.table.pqt')

In [ ]:
for dset in dsets:
    if 'trials' in dset:
        print(dset)
        

In [ ]:
for eid in eids:
    behavior = one.load_dataset(eid, 'trials')

In [18]:
from one.api import ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')


In [22]:
# remove log directory
import shutil
log_folder = os.path.join(os.getcwd(), 'logs')
if os.path.isdir(log_folder):
    shutil.rmtree(log_folder)

In [21]:
pid = '8185f1e9-cfe0-4fd6-8d7e-446a8051c588'
one.list_datasets(pid)

,,file_size,hash,default_revision,qc,exists_flatiron,exists_aws,exists,session_path,rel_path
eid,id,,,,,,,,,


In [13]:
print(one.alyx.user)

intbrainlab


In [10]:
one.list_datasets(eid, collection='alf')

[]

In [16]:
eid, probe_label = job_args[25]
print(f"Running job for {eid} and {probe_label}")

data_extraction_with_params(eid, probe_label)

Running job for e26c6001-defe-42a9-9ded-368e3f03ac61 and probe00
eid: e26c6001-defe-42a9-9ded-368e3f03ac61
probe_label: probe00
kwargs: {'extract_wheel': True, 'extract_dlc': True, 'extract_spikes': True, 'extract_lfp': True, 'overwrite_behavior': False, 'overwrite_wheel': False, 'overwrite_dlc': False, 'overwrite_spikes': False, 'overwrite_lfp': False, 'resampled_fs': 500}
Extracting data for eid e26c6001-defe-42a9-9ded-368e3f03ac61 and probe 8185f1e9-cfe0-4fd6-8d7e-446a8051c588...
Extracting behavior data...


ALFObjectNotFound: Dataset "alf/_ibl_trials.table.pqt" not found 
 The ALF object was not found.  This may occur if the object or namespace or incorrectly formatted e.g. the object "_ibl_trials.intervals.npy" would be found with the filters `object="trials", namespace="ibl"` 

In [5]:
from one.api import ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')
from detect_bad_channels import detect_bad_channels
import os 
from config import  BRAIN_REGION_PAIRS, SUBMITIT_PARAMS, extraction_parameters
import pandas as pd
import numpy as np
import mne
from brainbox.io.one import SpikeSortingLoader
from iblatlas.atlas import AllenAtlas
from iblatlas.regions import BrainRegions
import ibldsp.voltage as voltage
from pprint import pprint

In [3]:
from one.api import ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')

In [ ]:
import os 
from config import  BRAIN_REGION_PAIRS, SUBMITIT_PARAMS, extraction_parameters
import pandas as pd
import numpy as np
import mne
import submitit
import gc
import config
import time
from functools import partial

from data_extraction import data_extraction


data_extraction_with_params = partial(data_extraction, **extraction_parameters)
##########################
# Submit jobs
##########################

if len(BRAIN_REGION_PAIRS) >1 :
    raise ValueError("BRAIN_REGION_PAIRS should have only one pair of brain regions for extraction")

region1 = BRAIN_REGION_PAIRS[0][0]
region2 = BRAIN_REGION_PAIRS[0][1]

table_path = os.path.join(os.getcwd(), f'eid_probe_info_{region1}_{region2}.csv')
if not os.path.isfile(table_path):
    print(f"eid_probe_info_{region1}_{region2}.csv does not exist at {os.getcwd()}")
    print("use eid_probe_info_table.py to generate the table")
    raise ValueError(f"table does not exist at {os.getcwd()}")

# Load the table
df = pd.read_csv(table_path)
job_args = []

for i, row in df.iterrows():
    if i>0:
        break
    eid = row['eid']
    probe_label_1 = row[f'label_{region1}'][2:-2]
    probe_label_2 = row[f'label_{region2}'][2:-2]
    probe_labels = [probe_label_1, probe_label_2]
    probe_labels = list(set(probe_labels))
    

    for probe_label in probe_labels:
        job_args.append((eid, probe_label))


print(f'Number of jobs to submit: {len(job_args)}')
log_dir  = config.log_dir
max_jobs_parallel = config.maxjobs
print(f'Maximum number of jobs to run in parallel: {max_jobs_parallel}')
executor = submitit.AutoExecutor(folder=log_dir)
executor.update_parameters(**config.SUBMITIT_PARAMS)

In [18]:
from one.api import ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')
eid = '5c0c560e-9e1f-45e9-b66e-e4ee7855be84'
pids, labels = one.eid2pid(eid)
probe_label = 'probe01'

pid = next((pid for pid, label in zip(pids, labels) if label == probe_label), None)

In [ ]:
print(zip(pids[0], pids[1]))